In [1]:
! pip install langchain tiktoken pypdf chromadb
! pip install transformers InstructorEmbedding sentence_transformers==2.2.2
! pip install accelerate bitsandbytes xformers
! pip install plum-dispatch==1.7.4 aspose.words Spire.PDF
# ! pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 22.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 89.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.6 MB/s e

In [2]:
# Loading necessary modeules
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceBgeEmbeddings
import aspose.words as aw
from spire.pdf.common import *
from spire.pdf import *
import os
import pandas as pd
import numpy as np

In [ ]:
# Loading Pdf
dir_name = "/kaggle/input/thermal-pdfs"
file_loader = DirectoryLoader(dir_name, glob="./*.pdf", loader_cls=PyPDFLoader)
documents = file_loader.load()

In [ ]:
# Removing New line Characters
for doc in documents:
    doc.page_content = doc.page_content.replace("\n","")

In [ ]:
# intializing TextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [ ]:
#splitting Documents to chunks
texts = text_splitter.split_documents(documents)

In [ ]:
# Function Extract Table from document and return prompt converted to document
def extract_dataframe_list(file_path, temp_dir):
    page = []
    try:
        file_extention = file_path[file_path.rindex('.'):]
        df_list = []
        if(file_extention in [".xlsx", ".csv", ".html", ".pdf"]):
            if(file_extention == ".html"):
                df_list = pd.read_html(file_path)
            elif(file_extention == ".csv"):
                df_list.append(pd.read_csv(file_path))
            elif(file_extention == ".xlsx"):
                df_list.append(pd.read_excel(file_path))
            elif(file_extention == ".pdf"):
                pdf = PdfDocument()
                pdf.LoadFromFile(file_path)
                total_pages = pdf.Pages.Count
                pdf.Split('/kaggle/working/temp_file_{0}.pdf')
                for i in range(total_pages):
                    temp_html_name = f'/kaggle/working/temp_file_{i}.html'
                    temp_pdf_name = f'/kaggle/working/temp_file_{i}.pdf'

                    doc = aw.Document(temp_pdf_name)
                    os.remove(temp_pdf_name)
                    doc.save(temp_html_name)
                    

                    try:
                        ls = pd.read_html(temp_html_name)
                        os.remove(temp_html_name)
                        df_list.extend(ls)
                        for j in range(len(ls)):
                            page.append(i+1)
                    except:
                        print('-',end='')
                for filename in os.listdir(temp_dir):
                    if os.path.isfile(os.path.join(temp_dir, filename)):
                        os.remove(os.path.join(temp_dir, filename))

        return df_list, page
    except:
        return [], page

def is_incremental(s1, s2):
    s1_ascii, s2_ascii = sum([ord(ele) for ele in s1]), sum([ord(ele) for ele in s2])
    if(s1_ascii+1 == s2_ascii or s1_ascii == s2_ascii):
        return True        
    else:
        return False

def check_incremental_row(tab_1, tab_2):
    # If first column incremental
    tab_1_last = tab_1.iloc[-1, 0]
    tab_2_first = tab_2.iloc[0, 0]
    tab_2_1st_col_name = tab_2.columns[0]
    
    # IF TAB1(1,LAST) == TAB2(1,1)
    if(is_incremental(str(tab_1_last), str(tab_2_first))):
        tab_1.loc[len(tab_1)] = tab_2.values[0]
        tab_2.columns = tab_1.columns
        return True
    # IF TAB1(1,LAST) == COL_NAME(1)
    elif(is_incremental(str(tab_1_last), str(tab_2_1st_col_name))):
        tab_2.columns
        tab_1.loc[len(tab_1)] = tab_2.columns
        tab_2.columns = tab_1.columns

        return True
    else:
        return False

def merge_same_column(df):
    i=1
    while i < len(df.columns):
        if(df.columns[i-1] == df.columns[i]):
            for j in df.iterrows():
                if (j[1][0]) !=j[1][1]:
                    val = f"{j[1][0]}"+f"{j[1][1]}"
                    j[1][0] = j[1][1]= val

            df = df.loc[:,~df.columns.duplicated()].copy()

        else:

            same_in_both_column = 0
            for j in df.iterrows():
                try:
                    if (j[1,0]) ==j[1,1]:
                        same_in_both_column+=1
                except:
                    continue
            if(same_in_both_column == df.shape[0]):
                val = f"{df.columns[i-1]}"+f"{df.columns[i]}"
                temp_dict = {df.columns[i-1]:val, df.columns[i] : val}
                df.rename(columns=temp_dict, inplace=True)
            else:
                i+=1
            df = df.loc[:,~df.columns.duplicated()].copy()

    return df

def filter_table(df_list, page):
    # filter individual table
    filtered_pages = []
    clean_tables = []
    for i,df in enumerate(df_list):
        
#         df = df.fillna("")
        merge_same_column(df)
        
        total_row = df.shape[0]
        total_column = df.shape[1]

        # to specify the column name if named [0,1,2,3...]
        if (df.columns == range(total_column)).all():
            updated_column_names = dict(zip(df.columns, df.iloc[0]))

            # Drop the first row since it's used for column renaming
            df = df.drop(0)
            df.reset_index(drop=True, inplace=True)
            total_row -= 1

            # Rename columns using the updated_column_names dictionary
            df.rename(columns=updated_column_names, inplace=True)

       # Delete rows if all elements are the same
        all_same_rows = df.apply(lambda row: row.nunique() != 1, axis=1)
        df = df[all_same_rows]

        # Delete columns if column name is null or if all elements are null
        df = df.dropna(axis=1, how='all')
        
        # delete columns if column name is nan
        df = df.drop(columns=[col for col in df.columns if col is np.nan])

        df = df.fillna("")
        df.index = range(len(df.index))
        try:
            if((df.columns == [df.columns[0]]*len(df.columns)).all()):
                df.columns = df.iloc[0]
                df.drop(df.index[0], inplace=True)
                df.reset_index(drop=True, inplace=True)
            else:
                df = merge_same_column(df)
        except:
            print('error')
        filtered_pages.append(page[i])
        clean_tables.append(df)
    
    # merge tables
    i=1
    while i <len(clean_tables):
        try:
            
            if clean_tables[i].shape[1] == clean_tables[i-1].shape[1]:
                # IF NO OF COLUMN SAME
                if((clean_tables[i].columns == clean_tables[i-1].columns).all()):

                # JOIN IF COLUMN NAME SAME
                    clean_tables[i-1] = pd.concat([clean_tables[i-1], clean_tables[i]])
                    clean_tables.pop(i)
                    filtered_pages.remove(filtered_pages[i])
                    
                elif(check_incremental_row(clean_tables[i-1], clean_tables[i])):
                    clean_tables[i-1] = pd.concat([clean_tables[i-1], clean_tables[i]])
                    clean_tables.pop(i)
                    filtered_pages.remove(filtered_pages[i])
                else:
                    i+=1
                
            else:
                i+=1
        except:
            i+=1
#     fcl = []
#     for df in clean_tables:
#         # delete single row column table
# #         if (df.shape[0] <= 1 or df.shape[1] <=1):
# #             continue
# # #         elif()
# #         else:
#         fcl.append(df)
            
    return clean_tables, filtered_pages

def give_prompt_from_document(file_path):
    extracted_dataframes, pages = extract_dataframe_list(file_path, '/kaggle/working/')
    
    filtered_tables, filtered_pages = filter_table(extracted_dataframes, pages)
#     print(len(filtered_tables), len(filtered_pages))
    
    
    doc_list = []
    for x,df in enumerate(filtered_tables):
        df.reset_index(drop=True, inplace=True)
        
        des_list = []
        for i in range(df.shape[0]):
            description = f"For {i} "
            row_values = [f"{df.columns[j]} is {df.iloc[i,j]}" for j in range(df.shape[1])]
            null_element_in_row = sum(1 for value in row_values if value == "")
            description += ", ".join(row_values) + "."
            des_list.append(description)
        doc_text = ''
        for i,des in enumerate(des_list):
            if len(doc_text+des) < 1200:
                doc_text+=des
                if i == len(des_list)-1:
                    doc =  Document(page_content=doc_text, metadata={"source": file_path, "page": filtered_pages[x]})
                    doc_list.append(doc)
            else:
                doc =  Document(page_content=doc_text, metadata={"source": file_path, "page": filtered_pages[x]})
                doc_list.append(doc)
                doc_text=des
                
#         prompt = "\n\n".join(des_list)
#         doc =  Document(page_content=prompt, metadata={"source": file_path, "page": filtered_pages[x]})
#         doc_list.append(doc)
#         final_prompt += "\n\n" + prompt
    
        
    

    return doc_list, filtered_tables

In [ ]:
doc_list = []
for filename in os.listdir(dir_name):
    if os.path.isfile(os.path.join(dir_name, filename)):
        print(os.path.join(dir_name, filename))
        doc_list.extend(give_prompt_from_document(os.path.join(dir_name, filename)))
texts.extend(doc_list)

In [3]:
#Loading Embedding Models
model_name = "BAAI/bge-large-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [4]:
# Directory To Store Embedding
persist_directory = "thermal"

In [ ]:
# Storing Embeddings in ChromaDB
vector_db = Chroma.from_documents(documents=texts,
                                  embedding=bge_embeddings,
                                persist_directory=persist_directory)
vector_db.persist()

In [5]:
vector_db = Chroma(persist_directory=persist_directory, embedding_function=bge_embeddings)

In [15]:
# intializing Document Retriver
retriever = vector_db.as_retriever(search_type="mmr",search_kwargs={"k":7})

In [7]:
# #Load LLM model 
# from transformers import AutoTokenizer, AutoModelForCausalLM, MistralForCausalLM
# import torch

# tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
# model = MistralForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta",
#                                             load_in_8bit=True,
#                                               device_map='auto',
#                                               torch_dtype=torch.float16,
#                                               low_cpu_mem_usage=True,
# #                                               attn_implementation="flash_attention_2",
#                                               )

from transformers import AutoTokenizer, AutoModelForCausalLM, MistralForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("WizardLM/WizardLM-13B-V1.2")
model = AutoModelForCausalLM.from_pretrained("WizardLM/WizardLM-13B-V1.2",
                                             pretraining_tp=1,
                                            load_in_8bit=True,
                                              device_map='auto',
                                              torch_dtype=torch.float16,
                                              low_cpu_mem_usage=True,
#                                               attn_implementation="flash_attention_2",
                                              )

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/26.0G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuratio

In [ ]:
# from huggingface_hub import login
# login()

In [8]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.9,
    top_k=50,
    top_p=0.6
)
local_llm = HuggingFacePipeline(pipeline = pipe)

2024-04-12 07:39:39.824368: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 07:39:39.824473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 07:39:39.954142: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# # Google Gemma
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")
# model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it",
#                                             load_in_8bit=True,
#                                               device_map='auto',
#                                               torch_dtype=torch.float16,
#                                               low_cpu_mem_usage=True,
#                                               )

In [ ]:
# # HuggingFace Pipeline for Model
# from transformers import pipeline
# from langchain.llms import HuggingFacePipeline
# pipe = pipeline(
#     "text-generation", 
#     model=model, 
#     tokenizer=tokenizer,
#     max_new_tokens=300,
#     do_sample=True,
#     temperature=0.8,
#     top_k=50,
#     top_p=0.95
# )
# local_llm = HuggingFacePipeline(pipeline = pipe)

In [ ]:
local_llm("Hello, can you help me?")

In [ ]:
# # Prompt Template

# from langchain.prompts import PromptTemplate
# template = '''<|system|>Use the following pieces of information to answer the user's question in a concise manner.
# If you don't know the answer, just say that you don't know, don't try to make up an answer.
# If answer is provided in the context, also report the Page Number.</s>

# <|user|>
# Context: {context}
#     Question: {question}</s>
# <|assistant|>
# '''
# prompt = PromptTemplate(input_variables=["context", "question"], template=template)

In [ ]:
# # prompt Template for Gemma
# from langchain.prompts import PromptTemplate
# template = '''You are a helpful, respectful, and honest assistant.
# Always answer as helpfully as possible using the context text provided.
# Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make sense or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Just say I don't know

# BEGIN CONTEXT
# {context}
# END CONTEXT

# Your turn:
# Question: {question}
# '''
# prompt = PromptTemplate(input_variables=["context", "question"], template=template)

In [9]:
from langchain.prompts import PromptTemplate
template = '''A chat between a curious user and an artificial intelligence assistant.

USER:
DOCUMENT: {context}

Question: {question}

Answer the users QUESTION using the DOCUMENT text above.
Keep your answer ground in the facts of the DOCUMENT and keep your answer as Small as possible.
If the DOCUMENT doesn’t contain the answer the Question, just say you cannot provide an Answer.

ASSISTANT:
'''
prompt = PromptTemplate(input_variables=["context", "question"], template=template)

In [10]:
question="What is the capacity of each unit in thermal power project?"

In [ ]:
# docs_result = retriever.get_relevant_documents(question)
# docs_text = "\n".join([f"Page Number:{doc.metadata['page']}\n{doc.page_content}" for doc in docs_result])
# query = prompt.format(question=question, context=docs_text)
# chat = [
#     { "role": "user", "content": query },
# ]
# tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

In [16]:
# Fuction take question as input returns context and llm output

def resolve_query(question):
    docs_result = retriever.get_relevant_documents(question)
    docs_text = "\n".join([f"Page Number:{doc.metadata['page']}\n{doc.page_content}" for doc in docs_result])
    query = prompt.format(question=question, context=docs_text)
#     print(query)
#     chat = [
#          { "role": "user", "content": query },
#     ]
#     query = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    ans = local_llm.invoke(query)
    return docs_text, ans


In [17]:
print(resolve_query(question)[1].split("ASSISTANT:\n")[-1])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


The capacity of each unit in the thermal power project is 500MW or above.


In [13]:
# Loading Google sheets of Questions
import pandas
df = pandas.read_csv("https://docs.google.com/spreadsheets/d/1ST0SwFtyx_-qIazYBZ1C8Z4JIDhHKX4rKNe6xkeUwkg/export?format=csv")

In [14]:
df.head()

,Q. No.,Section,Question,Chat gpt-C- Expected answer,Zephyr 7B beta Answers at temp=0.8,Zephyr 7B beta Answers at temp=0.8 Cosine Similarity,Zephyr 7B beta Answers at temp=0.7,Zephyr 7B beta Answers at temp=0.7 Cosine Similarity,Wizard 7B Answers,Wizard 7B Answers Cosine Similarity,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1,1.1,What is the capacity of each unit in thermal p...,Each unit has a capacity of 500 MW or above.,The capacity of each unit in the thermal power...,0.796418,The capacity of each unit in the thermal power...,0.757805,"I apologize, but I am unable to provide an ans...",0.571888,numerical: blue,"list,types,components,materials : yellow",what: red
1,2,1.1,How should bidders handle items and equipment ...,Bidders should include items and equipment not...,Bidders should include all items and equipment...,0.838390,According to the information provided in the c...,0.875549,"Answer:According to the given context, if the ...",0.849363,"theoretical,purpose - green",how: orange,fullform: grey
2,3,1.1.1,How long is the equipment performance guarante...,The equipment performance is guaranteed for a ...,The equipment performance is guaranteed for on...,0.862546,The equipment performance is guaranteed for a ...,0.814159,The answer to your question depends on the spe...,0.595440,NaN,NaN,NaN
3,4,1.1.1,What is included in the detailed design accord...,Detailed design includes the detailed design o...,"According to Page Number:26, the detailed desi...",0.790731,"According to page 26, the detailed design incl...",0.791598,The detailed design according to the scope of ...,0.802668,NaN,NaN,NaN
4,5,1.2.1.7,List the components of the superheater and reh...,The components include pneumatically actuated ...,Each desuperheating spray system shall essenti...,0.743205,Each desuperheating spray system for superheat...,0.769823,Answer:The components of the superheater and r...,0.770304,NaN,NaN,NaN


In [18]:
# Testing all Question

from tqdm import tqdm
from time import sleep
context = []
zephyr_answers = []
for row in tqdm(df.iterrows()):
    if True:
#         print(row[1]['Question'])
#         break
        answer = resolve_query(row[1]['Question'])[1].split("ASSISTANT:\n")[-1]
#         context.append(cont)
        zephyr_answers.append(answer)

0it [00:00, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1it [00:08,  8.79s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2it [00:29, 15.96s/it]

...

299it [2:04:09, 35.11s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

300it [2:05:02, 40.35s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

301it [2:05:18, 24.98s/it]


In [ ]:
# zephyr_answers

In [19]:
zephyr_series = pandas.Series(zephyr_answers, name="WizardLM 13B beta Answers at temp=0.6", index=range(len(zephyr_answers)))
# context_series = pandas.Series(context, name="Context Retrival",index=range(len(context)))

In [20]:
updated_df = pandas.concat([df,zephyr_series], axis=1)
updated_df

,Q. No.,Section,Question,Chat gpt-C- Expected answer,Zephyr 7B beta Answers at temp=0.8,Zephyr 7B beta Answers at temp=0.8 Cosine Similarity,Zephyr 7B beta Answers at temp=0.7,Zephyr 7B beta Answers at temp=0.7 Cosine Similarity,Wizard 7B Answers,Wizard 7B Answers Cosine Similarity,Unnamed: 10,Unnamed: 11,Unnamed: 12,WizardLM 13B beta Answers at temp=0.6
0,1,1.1,What is the capacity of each unit in thermal p...,Each unit has a capacity of 500 MW or above.,The capacity of each unit in the thermal power...,0.796418,The capacity of each unit in the thermal power...,0.757805,"I apologize, but I am unable to provide an ans...",0.571888,numerical: blue,"list,types,components,materials : yellow",what: red,The capacity of each unit in the thermal power...
1,2,1.1,How should bidders handle items and equipment ...,Bidders should include items and equipment not...,Bidders should include all items and equipment...,0.838390,According to the information provided in the c...,0.875549,"Answer:According to the given context, if the ...",0.849363,"theoretical,purpose - green",how: orange,fullform: grey,Bidders should handle items and equipment not ...
2,3,1.1.1,How long is the equipment performance guarante...,The equipment performance is guaranteed for a ...,The equipment performance is guaranteed for on...,0.862546,The equipment performance is guaranteed for a ...,0.814159,The answer to your question depends on the spe...,0.595440,NaN,NaN,NaN,The equipment performance is guaranteed for 12...
3,4,1.1.1,What is included in the detailed design accord...,Detailed design includes the detailed design o...,"According to Page Number:26, the detailed desi...",0.790731,"According to page 26, the detailed design incl...",0.791598,The detailed design according to the scope of ...,0.802668,NaN,NaN,NaN,\nThe detailed design of all equipment and equ...
4,5,1.2.1.7,List the components of the superheater and reh...,The components include pneumatically actuated ...,Each desuperheating spray system shall essenti...,0.743205,Each desuperheating spray system for superheat...,0.769823,Answer:The components of the superheater and r...,0.770304,NaN,NaN,NaN,The components of the superheater and reheater...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,297,2.2.4.1(xii-b),What maintenance facilities are required for t...,Hinged access doors shall be provided in the h...,Page Number: 137\n \n Maintenance facilities r...,0.684777,Page Number: 137\n \n Maintenance facilities r...,0.684777,Answer:The following maintenance facilities ar...,0.728156,NaN,NaN,NaN,The maintenance facilities required for the ai...
297,298,2.2.4.1(xiii),What firefighting facilities are required for ...,Permanent firefighting facilities along with t...,Permanent firefighting facilities along with t...,0.846784,"Permanent firefighting facilities, including s...",0.902515,The firefighting facilities required for the a...,0.810368,NaN,NaN,NaN,Firefighting facilities are required for the a...
298,299,2.2.4.2(ii),Where should the condensate from the SCAPH be ...,Condensate from the SCAPH shall be led to the ...,The condensate from the SCAPH shall be led to ...,0.928949,Page Number: 210\n Answer: The condensate from...,0.770248,The condensate from the SCAPH can be led to th...,0.785042,NaN,NaN,NaN,\nCondensate from the Steam Coil Air Pre-heate...
299,300,2.2.4.2(v),What facility should be provided for the SCAPH?,Facility shall be provided for complete isolat...,Page Number:138\n The facility for complete is...,0.876397,Page Number:138\n Facility should be provided ...,0.823458,Answer: Based on the given technical specifica...,0.790424,NaN,NaN,NaN,The facility that should be provided for the S...


In [21]:
updated_df.to_csv("answers1.csv")

In [22]:
updated_df.loc[0,"Zephyr 7B beta Answers"]

KeyError: 'Zephyr 7B beta Answers'

In [ ]:
print(resolve_query("What is the capacity of each unit in the thermal power project?")[1])

In [24]:
from tqdm import tqdm
from time import sleep
from chromadb.utils.distance_functions import cosine
cosine_similaritys = []
for row in tqdm(updated_df.iterrows()):
    chatgpt_ans, wizardlm_ans = row[1]['Chat gpt-C- Expected answer'], row[1]['WizardLM 13B beta Answers at temp=0.6']
    chatgpt_embedd, wizardlm_embedd = bge_embeddings.embed_query(chatgpt_ans), bge_embeddings.embed_query(wizardlm_ans)
    cosine_similarity = cosine(chatgpt_embedd, wizardlm_embedd)
    cosine_similaritys.append(cosine_similarity)
    

0it [00:00, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

300it [00:30,  9.57it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

301it [00:30,  9.85it/s]


In [27]:
cosine = pandas.Series(cosine_similaritys, name="Cosine Similarity WizardLM 13B", index=range(len(cosine_similaritys)))

In [28]:
updated_df = pandas.concat([updated_df,cosine], axis=1)

In [29]:
updated_df.to_csv("cosine.csv")

In [30]:
cosine

0      0.092537
1      0.087839
2      0.003845
3      0.155193
4      0.211972
         ...   
296    0.320863
297    0.098516
298    0.087598
299    0.244360
300    0.114987
Name: Cosine Similarity WizardLM 13B, Length: 301, dtype: float64